<a href="https://colab.research.google.com/github/erarah/techtalk/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5135429%2F8586093%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240629%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240629T234043Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D46cefd396b6763364dc6deaba3b940280520b0448a97ecdf3860b9888b850bf06035cd42712ee780f98f94f8fbc8d79929e53eb5de40da79a8e47d4a65fbb61881200a9a9ff2cfc0b92301d422de9d72c588e1014cb282d835ead6a3fc440a4ba47226091991bb9974e63bcb2933110344960b20afc40b9868829b302e9d03d049fd1b0f79d1d42d5b01bb705bc90ca06ca09d17e110d7410c11c3e0f1cafe5361538e956856e14801c3d47a5546e8c787bf63b9f2d20d43dd43433aa63309d9597da2a3ffe6c0c43c4b5255b9ec26b9f6d19f6cd1cda01cfd956bc01c322718cdc3118a489a4be96b1ea35a9af5e294663606b3743d0f006931f7506cf09fb4,utilities:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5142984%2F8596988%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240629%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240629T234043Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8520be230fb9197aa702a8781f5b31842fe6d4a3b51f32db3deeb33964b3da7f7571a2ad30df9af382fea37490337de0fb938c604e8ac57f204e266f498d694646b0ed8458ff9deae5d68588861fd1997c6b2dd6096fcb2bf3f24fe515f97c7ebb1ab4ccd572ff67ea0e42012b73d3bc5a231cd00c7d02a82ac6c25be06bc7f021f7d95167f1a2786ee33faafe1f2949f781057ac62d10b5efed3e8a892d25df2589cfd66b61c3a7d432be6747cd7e69586fcf4f715aee2f3cf0363eaf64b9d32247b8fb0aca07c62ee4ecbbf7e708d681de9aa42519adb5f429e667d91dce00cc4db1db54c65fad62ae55e01d1804a97fce819de48869a92a92ef8b5b3bedbc,course-genres:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5143385%2F8597310%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240629%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240629T234043Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2f7d7ccbe8f9a31941178c4203b408041fa56e7eb71003293f41f77d081539b18ccde27a5d63dc1492b53eb92196d41ab0433f835171a27f9f97bca0f1f05ecdacb0e167b7be142be088905ded79a6162358460845dee6e09c148352552276af1df50cb9d67329510e5c62c436540b986078d106836ff4eea1088751f19c906fe7c466250a38ca0e51fde9b530d49f23f6aef88fadb5142272cde54ed002574d16444914b2cbc3a356de389cbd92ca692480166bfec4934507ff5285b7171d1e52e638608972f27c8ebc48eb75808f4d058ca7a042286b18e9ead50e42ca5993f1e6af6ee4bc8d7e77edc04dd63f4b140861d7e8dba33106ea33c34fa37ce933'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, sys

In [ ]:
sys.path.append('/kaggle/input/utilities')
from plots import *

In [ ]:
ratings_df = pd.read_csv('/kaggle/input/ratings/ratings.csv')
ratings_df.head()

In [ ]:
genres_df = pd.read_csv('/kaggle/input/course-genres/course_genre.csv')
genres_df.head()

In [ ]:
genres = genres_df.columns[2:].tolist()
genres

In [ ]:
counts = genres_df[genres].sum()
total = sum(counts)
titles = []
for genre in genres:
    title = ''
    prev = 0
    for i, c in enumerate(genre):
        if i > 0:
            if c.isupper():
                title += genre[prev:i] + ' '
                prev = i
    if title == '':
        titles.append(genre)
    else:
        title += genre[prev:]
        titles.append(title)

In [ ]:
plot_data_h(counts,
            titles,
            fig_size=(16,8),
            total=total)

In [ ]:
plot_data_h(counts,
            titles,
            fig_size=(16,8),
            total=total,
            file_name='genre_distribution')

In [ ]:
enrollments = ratings_df.groupby('user')['item'].count()

course_enrollments = ratings_df.groupby('item')['user'].count()
courses_df = pd.DataFrame(course_enrollments)
courses_df.sort_values(by='user', ascending=True, inplace=True)

courses_descr_df = pd.merge(courses_df, genres_df[['COURSE_ID','TITLE']], how='left', left_on='item', right_on='COURSE_ID')
courses_descr_df['TITLE'] = courses_descr_df['TITLE'].apply(lambda x : x.title())
courses_descr_df.rename(columns={'user':'Enrollments', 'TITLE':'Title', 'COURSE_ID':'Course'}, inplace=True)
courses_descr_df = courses_descr_df[['Course','Title','Enrollments']]

hist, bins = np.histogram(enrollments, bins=30)

In [ ]:
def plot_enrollment_histogram(values, total, preview=True):
    bar_width = 1
    fig = plt.figure(figsize=(16,8))
    plt.tick_params(axis='y', which='both', left=False, labelleft=False)
    ax = fig.gca()
    if preview:
        fig.patch.set_facecolor('#808080')  #temp
        ax.set_facecolor('#808080')
    for pos in range(0,14000,2000):
        plt.axhline(y=pos, color='white', alpha=0.3, linestyle='-', linewidth=1)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    ax.set_xticks([i for i in range(len(hist))])
    ax.set_xticklabels([f'{b:.0f}' for b in bins[:-1]], fontdict=tick_font)

    ax.tick_params(axis='x', colors='white')

    bars = plt.bar([i for i in range(len(hist))], hist, color='cyan', width=bar_width)

    i=0
    _, offset = ax.transAxes.inverted().transform([0,20000])
    for bar, value in zip(bars, values):
        if value > 0:
            plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'\u00a0{value:.0f} ({value/total:.2%})', ha='center', va='bottom', color='white',  fontdict={'fontsize':10, 'fontweight':'bold'}, rotation=90); i+=1;

    plt.tight_layout()
    plt.savefig('user_histogram.svg', format='svg', transparent=True)
    plt.show()

In [ ]:
plot_enrollment_histogram(hist, enrollments.shape[0], True)

In [ ]:
plot_enrollment_histogram(hist, enrollments.shape[0], False)

In [ ]:
total =  courses_descr_df['Enrollments'].values.sum()
plot_data_h(courses_descr_df.iloc[-20:]['Enrollments'].values,
            courses_descr_df.iloc[-20:]['Title'].values,
            fig_size=(16,8),
            total=total)



In [ ]:
plot_data_h(courses_descr_df.iloc[-20:]['Enrollments'].values,
            courses_descr_df.iloc[-20:]['Title'].values,
            fig_size=(16,8),
            total=total,
            file_name='most_popular_courses')